This will use the ESA SNAP GPT tool to create GRD products from the already downloaded SLC imagery. With the GRD data, the Normalized Difference Polarization Index, VV/VH, VH/VV, Normalized VH index, and Normalized VV index will be used to create water masks. Otsu thresholding will be done to label pixels as subaqeuous or subaerial.

These labeled pixels will be used for two things:
1. Generating a water mask for the InSAR time series (maybe the lowest water extent)
2. Generating a time series of subaerial change for individual creation sites

Plan is to couple the InSAR and Area time series to estimate volumetric changes of wetland BUDM sites 

In [10]:
import os
import subprocess
from datetime import datetime

# Establish GPT path, number of processors, and memory usage

In [40]:
GPT_PATH = '/home/wcc/esa-snap/bin/gpt'
NUM_PROCESSORS = 26
MEMORY_SIZE = '96G'

In [38]:
cmd = f'{GPT_PATH} ThermalNoiseRemoval -h -c {MEMORY_SIZE}'

try:
    result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout.decode())
except subprocess.CalledProcessError as e:
    print(f"Command failed: {e.stderr.decode()}")

Usage:
  gpt ThermalNoiseRemoval [options] 

Description:
  Removes thermal noise from products


Source Options:
  -SsourceProduct=<file>    Sets source 'sourceProduct' to <filepath>.
                            This is a mandatory source.

Parameter Options:
  -PoutputNoise=<boolean>                               Output noise
                                                        Default value is 'false'.
  -PreIntroduceThermalNoise=<boolean>                   Re-introduce thermal noise
                                                        Default value is 'false'.
  -PremoveThermalNoise=<boolean>                        Remove thermal noise
                                                        Default value is 'true'.
  -PselectedPolarisations=<string,string,string,...>    The list of polarisations

Graph XML Format:
  <graph id="someGraphId">
    <version>1.0</version>
    <node id="someNodeId">
      <operator>ThermalNoiseRemoval</operator>
      <sources>
        <sourceProdu

# Get a list of the available SLC images downloaded in 01a_get_slc.ipynb

In [41]:
# Establish working directories, and locate Sentinel-1 SLC .zip files
proj_dir = '/home/wcc/Desktop/SabineRS'
work_dir = os.path.join(proj_dir, 'Sentinel-1')

In [42]:
# assuming you have downloaded .zip files covering your AOI from ASF Vertex
# enter the file directory below
slc_zips = os.path.join(work_dir, 'SLC/ASCENDING/136/93')

slc_zips_list = sorted(os.listdir(slc_zips), key=lambda x: datetime.strptime(x[17:25], '%Y%m%d'))
slc_zips_dirs = [os.path.join(slc_zips, slc) for slc in slc_zips_list]
slc_zips_dates = [slc[17:25] for slc in slc_zips_list]

# Establish GRD working directories

In [43]:
def project_dir(work_dir):
    """
    This function reads in a string that you wish to make your working directory 
    for the InSAR project, and creates a data directory to store the data for ISCE2 and mintpy
    work-dir = str
        path to the directory created in 01_get_slc.ipynb
    """

    #creates file on your desktop containing the work of this notebook
    os.makedirs(work_dir, exist_ok=True)
    
    # path for GRD data storage
    grd_dir = os.path.join(work_dir, 'GRD')
    os.makedirs(grd_dir, exist_ok=True)

    # list of grd directories
    grdpaths = []
    for step in ['0_tnr', '1_cal', '2_deburst', '3_multilook', '4_speckle', '5_spgr', '6_grd']:
        grdpaths.append(os.path.join(grd_dir, step))
        os.makedirs(os.path.join(grd_dir, step), exist_ok=True)


    return grd_dir, grdpaths

In [44]:
grd_dir, grdpaths = project_dir(work_dir)

# Processing steps for SLC to GRD:
1. Thermal Noise Removal
2. Calibration
3. TOPSAR-Deburst
4. Multilook
5. Speckle-Filter
6. SPGR
7. GRD
8. Write

1. Thermal Noise Removal

In [45]:
OUTPUT_NOISE = False
REINTRODUCE_TN = False
REMOVE_TN = True
POLARISATIONS = 'VV,VH'

for i, file in enumerate(slc_zips_list):
    OUTFILE = os.path.join(grdpaths[0], f'{slc_zips_dates[i]}.dim')
    tnr_cmd = f'{GPT_PATH} ThermalNoiseRemoval -SsourceProduct={os.path.join(slc_zips,file)} -PoutputNoise={OUTPUT_NOISE} -PreIntroduceThermalNoise={REINTRODUCE_TN} -PremoveThermalNoise={REMOVE_TN} -PselectedPolarisations={POLARISATIONS} -t {OUTFILE} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(tnr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

Executing operator...
20%....30%....40%....50%....60%....70%....80%....90%....100% done.
Writing...
...11%...21%....31%....41%....51%....61%....71%....81%....91%.... done.

Executing operator...
20%....30%....40%....50%....60%....70%....80%....90%....100% done.
Writing...
...11%...21%.....31%....41%....51%....61%....71%....81%....91%.... done.

Executing operator...
20%....30%....40%....50%....60%....70%....80%....90%....100% done.
Writing...
...11%...21%.....31%....41%....51%....61%....71%....81%....91%.... done.

Executing operator...
20%....30%....40%....50%....60%....70%....80%....90%....100% done.
Writing...
...11%...21%....31%....41%....51%....61%....71%....81%....91%.... done.

Executing operator...
20%....30%....40%....50%....60%....70%....80%....90%....100% done.
Writing...
...11%...21%....31%....41%....51%....61%....71%....81%....91%.... done.

Executing operator...
20%....30%....40%....50%....60%....70%....80%....90%....100% done.
Writing...
...11%...21%....31%....41%....51%

KeyboardInterrupt: 

2. Calibration

In [ ]:
OUTPUT_NOISE = False
REINTRODUCE_TN = False
REMOVE_TN = True
POLARISATIONS = 'VV,VH'

for i, file in enumerate(slc_zips_list):
    OUTFILE = os.path.join(grdpaths[0], f'{slc_zips_dates[i]}.dim')
    tnr_cmd = f'{GPT_PATH} ThermalNoiseRemoval -SsourceProduct={os.path.join(slc_zips,file)} -PoutputNoise={OUTPUT_NOISE} -PreIntroduceThermalNoise={REINTRODUCE_TN} -PremoveThermalNoise={REMOVE_TN} -PselectedPolarisations={POLARISATIONS} -t {OUTFILE} -c {MEMORY_SIZE} -q {NUM_PROCESSORS}'
    try:
        result = subprocess.run(tnr_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print(f"Command failed: {e.stderr.decode()}")

3. TOPSAR-Deburst

4. Multilook

5. Speckle-Filter

6. SPGR

7. GRD

8. Write